# 📊 Lezione 17 — Metriche Avanzate di Valutazione

## Obiettivi della lezione

In questa lezione imparerai:
- 🎯 **Perché Accuracy non basta** — il problema delle classi sbilanciate
- 📊 **Precision, Recall, F1-Score** — metriche per classificazione
- 🔢 **Matrice di Confusione** — capire TP, TN, FP, FN
- 📈 **Curva ROC e AUC** — valutare il modello a diverse soglie
- 📉 **Curva Precision-Recall** — alternativa per classi sbilanciate
- ⚙️ **Scelta della soglia** — ottimizzare per il business

---

## 📍 Posizione nel Corso

```
Lezione 15: Gradient Boosting (XGBoost, LightGBM)
Lezione 16: Feature Importance
👉 Lezione 17: Metriche Avanzate di Valutazione ← SEI QUI
Lezione 18: Clustering (K-Means, DBSCAN)
Lezione 19: Riduzione Dimensionalità (PCA)
```

---

## ❓ Il Problema dell'Accuracy

```
┌─────────────────────────────────────────────────────────────────┐
│                SCENARIO: Rilevamento Frodi                       │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│  Dataset: 10,000 transazioni                                    │
│     - 9,900 legittime (99%)                                     │
│     - 100 frodi (1%)                                            │
│                                                                  │
│  Modello "stupido": predice SEMPRE "legittima"                  │
│     → Accuracy = 99%! 🎉                                        │
│     → Ma non trova NESSUNA frode! 😱                            │
│                                                                  │
├─────────────────────────────────────────────────────────────────┤
│  MORALE: Accuracy è INGANNEVOLE con classi sbilanciate!         │
│          Servono metriche che catturino i veri obiettivi.       │
└─────────────────────────────────────────────────────────────────┘
```

# Section 1 — La Matrice di Confusione

---

## 🔢 I 4 Tipi di Predizioni

```
                        PREDIZIONE DEL MODELLO
                    ┌───────────────┬───────────────┐
                    │   POSITIVO    │   NEGATIVO    │
        ┌───────────┼───────────────┼───────────────┤
        │           │     TP        │     FN        │
 REALTÀ │ POSITIVO  │ True Positive │False Negative │
        │           │  (HIT! ✅)    │ (MISS! 😱)    │
        ├───────────┼───────────────┼───────────────┤
        │           │     FP        │     TN        │
        │ NEGATIVO  │False Positive │ True Negative │
        │           │(Falso allarme)│ (Corretto ✅) │
        └───────────┴───────────────┴───────────────┘

ESEMPIO MEDICO (test per malattia):
────────────────────────────────────
• TP: Malato, test positivo    → Diagnosi corretta!
• TN: Sano, test negativo      → Diagnosi corretta!
• FP: Sano, test positivo      → Falso allarme (ansia inutile)
• FN: Malato, test negativo    → PERICOLOSO! Malattia non rilevata
```

## 📊 Le Metriche Derivate

```
                    TP                          TP
PRECISION = ─────────────────      RECALL = ─────────────────
             TP + FP                         TP + FN
             
"Di quelli che ho detto            "Di tutti i veri positivi,
 positivi, quanti lo erano?"        quanti ne ho trovati?"
 
                    TN
SPECIFICITY = ─────────────────    (Recall sui negativi)
               TN + FP

              2 × Precision × Recall
F1-SCORE = ─────────────────────────   (Media armonica)
              Precision + Recall
```

In [ ]:
# ============================================
# DEMO: Matrice di Confusione
# ============================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import warnings
warnings.filterwarnings('ignore')

print("=" * 60)
print("MATRICE DI CONFUSIONE")
print("=" * 60)

# Creiamo un dataset sbilanciato (10% positivi)
X, y = make_classification(
    n_samples=1000,
    n_features=10,
    n_informative=5,
    weights=[0.9, 0.1],  # 90% classe 0, 10% classe 1
    random_state=42
)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(f"\n📊 Dataset sbilanciato:")
print(f"   Classe 0 (negativa): {(y == 0).sum()} ({(y == 0).mean():.1%})")
print(f"   Classe 1 (positiva): {(y == 1).sum()} ({(y == 1).mean():.1%})")

# Training modello
model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Matrice di confusione
cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cm.ravel()

print(f"\n🔢 Matrice di Confusione:")
print(f"   ┌─────────────────────────────────┐")
print(f"   │          PREDETTO               │")
print(f"   │      Neg          Pos           │")
print(f"   ├─────────────────────────────────┤")
print(f"   │ TN={tn:3d}        FP={fp:3d}    ← Reali Neg │")
print(f"   │ FN={fn:3d}        TP={tp:3d}    ← Reali Pos │")
print(f"   └─────────────────────────────────┘")

# Calcolo metriche manuale
accuracy = (tp + tn) / (tp + tn + fp + fn)
precision = tp / (tp + fp) if (tp + fp) > 0 else 0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0
f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

print(f"\n📈 Metriche calcolate:")
print(f"   Accuracy:    {accuracy:.4f}  ← (TP+TN) / Totale")
print(f"   Precision:   {precision:.4f}  ← TP / (TP+FP)")
print(f"   Recall:      {recall:.4f}  ← TP / (TP+FN)")
print(f"   Specificity: {specificity:.4f}  ← TN / (TN+FP)")
print(f"   F1-Score:    {f1:.4f}  ← Media armonica P e R")

# Visualizzazione
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Matrice confusione visuale
ax1 = axes[0]
disp = ConfusionMatrixDisplay(cm, display_labels=['Negativo', 'Positivo'])
disp.plot(ax=ax1, cmap='Blues', values_format='d')
ax1.set_title('📊 Matrice di Confusione')

# Metriche come bar chart
ax2 = axes[1]
metriche = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'Specificity']
valori = [accuracy, precision, recall, f1, specificity]
colors = ['#3498db', '#2ecc71', '#e74c3c', '#9b59b6', '#f39c12']
bars = ax2.bar(metriche, valori, color=colors, edgecolor='black', alpha=0.8)
ax2.set_ylim(0, 1.1)
ax2.set_ylabel('Valore')
ax2.set_title('📈 Metriche di Classificazione')
for bar, val in zip(bars, valori):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
             f'{val:.3f}', ha='center', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

# Section 2 — Precision vs Recall: Il Trade-off

---

## ⚖️ Non Puoi Avere Tutto!

```
┌─────────────────────────────────────────────────────────────────┐
│              PRECISION vs RECALL TRADE-OFF                       │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│  ALTA PRECISION, BASSO RECALL:                                  │
│  ─────────────────────────────                                  │
│  "Predico positivo SOLO quando sono sicurissimo"                │
│  → Pochi falsi positivi, ma perdo molti veri positivi           │
│  → Esempio: Approvare prestiti solo ai clienti perfetti         │
│                                                                  │
│  ALTO RECALL, BASSA PRECISION:                                  │
│  ─────────────────────────────                                  │
│  "Predico positivo anche se ho qualche dubbio"                  │
│  → Trovo quasi tutti i positivi, ma molti falsi allarmi         │
│  → Esempio: Test medico che non deve perdere nessun malato      │
│                                                                  │
└─────────────────────────────────────────────────────────────────┘
```

## 🎯 Quando Preferire Cosa?

| Scenario | Priorità | Perché |
|----------|----------|--------|
| **Diagnosi cancro** | RECALL | Un FN = malato non curato → GRAVE |
| **Filtro spam** | PRECISION | Un FP = email importante persa → fastidioso |
| **Rilevamento frodi** | Dipende | Bilanciare costi FP vs FN |
| **Ricerca web** | PRECISION | Meglio pochi risultati buoni che tanti inutili |
| **Sistema allerta terremoti** | RECALL | Meglio falsi allarmi che mancarne uno |

## 📊 F1-Score: La Soluzione?

```
F1 = 2 × (Precision × Recall) / (Precision + Recall)

• È la MEDIA ARMONICA di P e R
• Penalizza valori sbilanciati
• Se P=1.0 e R=0.1 → F1=0.18 (non 0.55!)
• Buon compromesso quando non sai cosa preferire
```

In [ ]:
# ============================================
# DEMO: Il Trade-off Precision-Recall
# Variando la soglia di classificazione
# ============================================

from sklearn.metrics import precision_recall_curve, classification_report

print("=" * 60)
print("TRADE-OFF PRECISION vs RECALL")
print("=" * 60)

# Otteniamo le probabilità invece delle classi
y_proba = model.predict_proba(X_test)[:, 1]  # Probabilità classe 1

print(f"\n📊 Distribuzione probabilità predette:")
print(f"   Min: {y_proba.min():.4f}")
print(f"   Max: {y_proba.max():.4f}")
print(f"   Media: {y_proba.mean():.4f}")

# Calcoliamo precision e recall per diverse soglie
precision_curve, recall_curve, thresholds = precision_recall_curve(y_test, y_proba)

# Visualizziamo l'effetto di diverse soglie
soglie_esempio = [0.3, 0.5, 0.7]

print("\n📈 Effetto della soglia:")
print("-" * 60)
print(f"{'Soglia':<10} {'Precision':<12} {'Recall':<12} {'F1-Score':<12}")
print("-" * 60)

risultati_soglie = []
for soglia in soglie_esempio:
    y_pred_soglia = (y_proba >= soglia).astype(int)
    
    tp = ((y_pred_soglia == 1) & (y_test == 1)).sum()
    fp = ((y_pred_soglia == 1) & (y_test == 0)).sum()
    fn = ((y_pred_soglia == 0) & (y_test == 1)).sum()
    
    prec = tp / (tp + fp) if (tp + fp) > 0 else 0
    rec = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * prec * rec / (prec + rec) if (prec + rec) > 0 else 0
    
    risultati_soglie.append({'Soglia': soglia, 'Precision': prec, 'Recall': rec, 'F1': f1})
    print(f"{soglia:<10.1f} {prec:<12.4f} {rec:<12.4f} {f1:<12.4f}")

# Visualizzazione
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Curva Precision-Recall
ax1 = axes[0]
ax1.plot(recall_curve, precision_curve, linewidth=2, color='#3498db')
ax1.fill_between(recall_curve, precision_curve, alpha=0.2, color='#3498db')

# Punti per le soglie esempio
for res in risultati_soglie:
    ax1.scatter(res['Recall'], res['Precision'], s=150, zorder=5, 
                label=f"Soglia={res['Soglia']}")

ax1.set_xlabel('Recall')
ax1.set_ylabel('Precision')
ax1.set_title('📉 Curva Precision-Recall')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_xlim([0, 1.05])
ax1.set_ylim([0, 1.05])

# Precision e Recall vs Soglia
ax2 = axes[1]
# Nota: precision_curve ha un elemento in più di thresholds
ax2.plot(thresholds, precision_curve[:-1], label='Precision', linewidth=2, color='#2ecc71')
ax2.plot(thresholds, recall_curve[:-1], label='Recall', linewidth=2, color='#e74c3c')

# Linea a soglia 0.5
ax2.axvline(x=0.5, color='gray', linestyle='--', linewidth=1.5, label='Soglia default (0.5)')

ax2.set_xlabel('Soglia di Classificazione')
ax2.set_ylabel('Valore')
ax2.set_title('📊 Precision e Recall vs Soglia')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 Osservazione:")
print("   - Soglia BASSA → alto Recall, bassa Precision (più permissivo)")
print("   - Soglia ALTA → alta Precision, basso Recall (più selettivo)")

# Section 3 — Curva ROC e AUC

---

## 📈 ROC = Receiver Operating Characteristic

```
COSA MOSTRA LA CURVA ROC:
─────────────────────────

Asse X: False Positive Rate (FPR) = FP / (FP + TN)
        "Quanti negativi ho sbagliato?"
        
Asse Y: True Positive Rate (TPR) = TP / (TP + FN) = RECALL
        "Quanti positivi ho trovato?"

Ogni punto sulla curva = una soglia diversa.

INTERPRETAZIONE:
────────────────

     TPR
     1.0 ┤        ╭──────── Modello perfetto
         │       ╱
         │     ╱
         │   ╱  ← Modello buono
         │ ╱
     0.5 │╱─────────────── Random (diagonale)
         │
     0.0 └──────────────────
         0.0    0.5    1.0
                FPR
```

## 🎯 AUC = Area Under the Curve

| AUC | Interpretazione |
|-----|-----------------|
| 1.0 | Modello perfetto |
| 0.9-1.0 | Eccellente |
| 0.8-0.9 | Buono |
| 0.7-0.8 | Discreto |
| 0.5-0.7 | Scarso |
| 0.5 | Random (inutile) |
| < 0.5 | Peggio di random! |

**AUC = Probabilità che il modello assegni una probabilità più alta
a un positivo casuale rispetto a un negativo casuale.**

In [ ]:
# ============================================
# DEMO: Curva ROC e AUC
# ============================================

from sklearn.metrics import roc_curve, roc_auc_score, RocCurveDisplay
from sklearn.ensemble import RandomForestClassifier

print("=" * 60)
print("CURVA ROC E AUC")
print("=" * 60)

# Confrontiamo più modelli
modelli = {
    'Logistic Regression': LogisticRegression(random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=-1)
}

# Visualizzazione
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Curva ROC per ogni modello
ax1 = axes[0]
colors = ['#3498db', '#2ecc71']

risultati_auc = []

for (nome, mod), color in zip(modelli.items(), colors):
    mod.fit(X_train, y_train)
    y_proba_mod = mod.predict_proba(X_test)[:, 1]
    
    # Calcolo ROC
    fpr, tpr, thresholds_roc = roc_curve(y_test, y_proba_mod)
    auc_score = roc_auc_score(y_test, y_proba_mod)
    
    risultati_auc.append({'Modello': nome, 'AUC': auc_score})
    
    # Plot
    ax1.plot(fpr, tpr, label=f'{nome} (AUC = {auc_score:.3f})', 
             linewidth=2, color=color)

# Linea random
ax1.plot([0, 1], [0, 1], 'k--', linewidth=1.5, label='Random (AUC = 0.5)')
ax1.fill_between([0, 1], [0, 1], alpha=0.1, color='gray')

ax1.set_xlabel('False Positive Rate (FPR)')
ax1.set_ylabel('True Positive Rate (TPR = Recall)')
ax1.set_title('📈 Curva ROC - Confronto Modelli')
ax1.legend(loc='lower right')
ax1.grid(True, alpha=0.3)

# Zoom sulla parte interessante
ax2 = axes[1]
for (nome, mod), color in zip(modelli.items(), colors):
    y_proba_mod = mod.predict_proba(X_test)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, y_proba_mod)
    auc_score = roc_auc_score(y_test, y_proba_mod)
    ax2.plot(fpr, tpr, label=f'{nome}', linewidth=2, color=color)

ax2.plot([0, 1], [0, 1], 'k--', linewidth=1.5)
ax2.set_xlim([0, 0.3])
ax2.set_ylim([0.7, 1.0])
ax2.set_xlabel('False Positive Rate (FPR)')
ax2.set_ylabel('True Positive Rate (TPR)')
ax2.set_title('🔍 Zoom: Angolo in Alto a Sinistra')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Risultati
print("\n📊 Confronto AUC:")
print("-" * 40)
for res in risultati_auc:
    quality = "⭐⭐⭐" if res['AUC'] > 0.9 else "⭐⭐" if res['AUC'] > 0.8 else "⭐"
    print(f"   {res['Modello']:25} AUC = {res['AUC']:.4f} {quality}")

print("\n💡 Interpretazione:")
print("   - AUC misura la capacità del modello di SEPARARE le classi")
print("   - Più la curva è vicina all'angolo in alto a sinistra, meglio è")

# Section 4 — ROC vs Precision-Recall: Quando Usare Cosa?

---

## ⚖️ Confronto tra le Due Curve

```
┌─────────────────────────────────────────────────────────────────┐
│                    ROC vs PRECISION-RECALL                       │
├──────────────────────────────┬──────────────────────────────────┤
│         CURVA ROC            │      CURVA PRECISION-RECALL      │
├──────────────────────────────┼──────────────────────────────────┤
│ Assi: FPR vs TPR             │ Assi: Recall vs Precision        │
│                              │                                  │
│ ✅ Buona per classi          │ ✅ Migliore per classi           │
│    bilanciate                │    SBILANCIATE                   │
│                              │                                  │
│ ⚠️ Può essere ottimistica    │ ✅ Più realistica su dataset     │
│    con classi sbilanciate    │    con molti negativi            │
│                              │                                  │
│ Baseline: diagonale          │ Baseline: linea orizzontale      │
│ (AUC = 0.5)                  │ (AP = % positivi)                │
│                              │                                  │
│ Metrica: AUC-ROC             │ Metrica: Average Precision (AP)  │
└──────────────────────────────┴──────────────────────────────────┘
```

## 🎯 Regola Pratica

| Situazione | Curva da Usare |
|------------|----------------|
| Classi bilanciate (~50/50) | ROC va bene |
| Classi sbilanciate (es. 95/5) | **Precision-Recall** |
| Confrontare modelli diversi | Entrambe! |
| Report per stakeholder non tecnici | ROC (più intuitiva) |
| Focus sulla classe minoritaria | **Precision-Recall** |

In [ ]:
# ============================================
# DEMO: Confronto ROC vs Precision-Recall
# Su dataset MOLTO sbilanciato
# ============================================

from sklearn.metrics import average_precision_score, PrecisionRecallDisplay

print("=" * 60)
print("ROC vs PRECISION-RECALL su Dataset Sbilanciato")
print("=" * 60)

# Creiamo un dataset MOLTO sbilanciato (2% positivi)
X_unbal, y_unbal = make_classification(
    n_samples=2000,
    n_features=10,
    n_informative=5,
    weights=[0.98, 0.02],  # 98% classe 0, 2% classe 1
    random_state=42
)

X_train_u, X_test_u, y_train_u, y_test_u = train_test_split(
    X_unbal, y_unbal, test_size=0.3, random_state=42
)

print(f"\n📊 Dataset MOLTO sbilanciato:")
print(f"   Classe 0 (negativa): {(y_unbal == 0).sum()} ({(y_unbal == 0).mean():.1%})")
print(f"   Classe 1 (positiva): {(y_unbal == 1).sum()} ({(y_unbal == 1).mean():.1%})")
print(f"   Test set positivi: {(y_test_u == 1).sum()}")

# Training modello
model_unbal = RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=-1)
model_unbal.fit(X_train_u, y_train_u)
y_proba_u = model_unbal.predict_proba(X_test_u)[:, 1]

# Calcolo metriche
fpr_u, tpr_u, _ = roc_curve(y_test_u, y_proba_u)
prec_u, rec_u, _ = precision_recall_curve(y_test_u, y_proba_u)

auc_roc_u = roc_auc_score(y_test_u, y_proba_u)
ap_u = average_precision_score(y_test_u, y_proba_u)

# Baseline per PR curve (proporzione di positivi)
baseline_pr = (y_test_u == 1).mean()

# Visualizzazione
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# ROC Curve
ax1 = axes[0]
ax1.plot(fpr_u, tpr_u, linewidth=2, color='#3498db', 
         label=f'Random Forest (AUC = {auc_roc_u:.3f})')
ax1.plot([0, 1], [0, 1], 'k--', linewidth=1.5, label='Random (AUC = 0.5)')
ax1.fill_between(fpr_u, tpr_u, alpha=0.2, color='#3498db')
ax1.set_xlabel('False Positive Rate')
ax1.set_ylabel('True Positive Rate')
ax1.set_title('📈 Curva ROC (sembra ottima!)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Precision-Recall Curve
ax2 = axes[1]
ax2.plot(rec_u, prec_u, linewidth=2, color='#e74c3c', 
         label=f'Random Forest (AP = {ap_u:.3f})')
ax2.axhline(y=baseline_pr, color='gray', linestyle='--', linewidth=1.5, 
            label=f'Baseline (= {baseline_pr:.3f})')
ax2.fill_between(rec_u, prec_u, alpha=0.2, color='#e74c3c')
ax2.set_xlabel('Recall')
ax2.set_ylabel('Precision')
ax2.set_title('📉 Curva Precision-Recall (più realistica!)')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_xlim([0, 1.05])
ax2.set_ylim([0, 1.05])

plt.tight_layout()
plt.show()

# Analisi
print("\n📊 Confronto Metriche:")
print("-" * 50)
print(f"   AUC-ROC:            {auc_roc_u:.4f}  ← Sembra eccellente!")
print(f"   Average Precision:  {ap_u:.4f}  ← Più realistica")
print(f"   Baseline PR:        {baseline_pr:.4f}")

print("\n💡 Osservazione CRITICA:")
print("   - ROC AUC = 0.95+ sembra fantastico...")
print("   - Ma Average Precision rivela che il modello fa ancora fatica")
print("   - Con classi sbilanciate, PR curve è più informativa!")
print("   - Il modello deve essere MOLTO meglio della baseline per essere utile")

# Section 5 — Scegliere la Soglia Ottimale

---

## ⚙️ Come Scegliere la Soglia di Classificazione?

```
Di default, i classificatori usano soglia = 0.5
Ma spesso NON è la scelta ottimale!

METODI PER SCEGLIERE LA SOGLIA:
───────────────────────────────

1️⃣  MASSIMIZZARE F1-SCORE
    → Trova il punto dove P e R sono bilanciate
    → Buon default quando non hai preferenze

2️⃣  MASSIMIZZARE YOUDEN'S J (per ROC)
    → J = TPR - FPR = Sensitivity + Specificity - 1
    → Trova il punto più lontano dalla diagonale

3️⃣  VINCOLO SUL RECALL (business-driven)
    → Es: "Voglio almeno il 90% di recall"
    → Trovi la soglia che garantisce quel recall

4️⃣  ANALISI COSTI/BENEFICI
    → Quantifichi il costo di FP vs FN
    → Minimizzi il costo totale
```

## 💰 Esempio: Rilevamento Frodi

```
COSTI:
• False Positive (blocco transazione legittima): -$10 (fastidio cliente)
• False Negative (frode non rilevata): -$1000 (perdita)

CALCOLO:
Costo Totale = FP × 10 + FN × 1000

→ Meglio MOLTI falsi allarmi che perdere una frode!
→ Soglia BASSA (es: 0.1) per massimizzare recall
```

In [ ]:
# ============================================
# DEMO: Trovare la Soglia Ottimale
# ============================================

from sklearn.metrics import f1_score

print("=" * 60)
print("TROVARE LA SOGLIA OTTIMALE")
print("=" * 60)

# Usiamo il modello sul dataset sbilanciato
y_proba_opt = model_unbal.predict_proba(X_test_u)[:, 1]

# Metodo 1: Massimizzare F1-Score
print("\n📌 Metodo 1: Massimizzare F1-Score")
print("-" * 50)

soglie_test = np.arange(0.05, 0.95, 0.05)
f1_scores = []

for soglia in soglie_test:
    y_pred_temp = (y_proba_opt >= soglia).astype(int)
    f1_temp = f1_score(y_test_u, y_pred_temp, zero_division=0)
    f1_scores.append(f1_temp)

best_f1_idx = np.argmax(f1_scores)
best_soglia_f1 = soglie_test[best_f1_idx]
best_f1 = f1_scores[best_f1_idx]

print(f"   Soglia ottimale per F1: {best_soglia_f1:.2f}")
print(f"   F1-Score massimo: {best_f1:.4f}")

# Metodo 2: Youden's J statistic (per ROC)
print("\n📌 Metodo 2: Youden's J (massimizza TPR - FPR)")
print("-" * 50)

fpr_opt, tpr_opt, thresholds_opt = roc_curve(y_test_u, y_proba_opt)
j_scores = tpr_opt - fpr_opt
best_j_idx = np.argmax(j_scores)
best_soglia_j = thresholds_opt[best_j_idx]

print(f"   Soglia ottimale (Youden's J): {best_soglia_j:.4f}")
print(f"   J statistic: {j_scores[best_j_idx]:.4f}")

# Metodo 3: Garantire minimo Recall
print("\n📌 Metodo 3: Garantire Recall ≥ 90%")
print("-" * 50)

target_recall = 0.90
for soglia in np.arange(0.01, 1.0, 0.01):
    y_pred_temp = (y_proba_opt >= soglia).astype(int)
    tp = ((y_pred_temp == 1) & (y_test_u == 1)).sum()
    fn = ((y_pred_temp == 0) & (y_test_u == 1)).sum()
    recall_temp = tp / (tp + fn) if (tp + fn) > 0 else 0
    
    if recall_temp >= target_recall:
        best_soglia_recall = soglia
        achieved_recall = recall_temp
        # Calcola precision a questa soglia
        fp = ((y_pred_temp == 1) & (y_test_u == 0)).sum()
        precision_at_recall = tp / (tp + fp) if (tp + fp) > 0 else 0
        break

print(f"   Soglia per Recall ≥ 90%: {best_soglia_recall:.2f}")
print(f"   Recall effettivo: {achieved_recall:.2%}")
print(f"   Precision a questa soglia: {precision_at_recall:.4f}")

# Visualizzazione
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# F1-Score vs Soglia
ax1 = axes[0]
ax1.plot(soglie_test, f1_scores, linewidth=2, color='#9b59b6', marker='o')
ax1.axvline(x=best_soglia_f1, color='red', linestyle='--', linewidth=2, 
            label=f'Ottimo: {best_soglia_f1:.2f}')
ax1.axvline(x=0.5, color='gray', linestyle=':', linewidth=1.5, label='Default: 0.5')
ax1.set_xlabel('Soglia')
ax1.set_ylabel('F1-Score')
ax1.set_title('📊 F1-Score vs Soglia')
ax1.legend()
ax1.grid(True, alpha=0.3)

# ROC con punto Youden
ax2 = axes[1]
ax2.plot(fpr_opt, tpr_opt, linewidth=2, color='#3498db')
ax2.scatter([fpr_opt[best_j_idx]], [tpr_opt[best_j_idx]], s=200, color='red', 
            zorder=5, label=f"Youden's J (soglia={best_soglia_j:.2f})")
ax2.plot([0, 1], [0, 1], 'k--', linewidth=1.5)
ax2.set_xlabel('False Positive Rate')
ax2.set_ylabel('True Positive Rate')
ax2.set_title("📈 Punto Ottimale su ROC (Youden's J)")
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Riepilogo
print("\n" + "=" * 60)
print("📋 RIEPILOGO SOGLIE OTTIMALI")
print("=" * 60)
print(f"""
   Soglia Default:         0.50
   Soglia Max F1:          {best_soglia_f1:.2f}
   Soglia Youden's J:      {best_soglia_j:.2f}
   Soglia Recall ≥ 90%:    {best_soglia_recall:.2f}
   
💡 La scelta dipende dal CONTESTO BUSINESS!
""")

# Section 6 — Schema Mentale

---

## 🧠 Quale Metrica Usare?

```
                    SCEGLI LA METRICA GIUSTA
                    ────────────────────────
                              │
                              ▼
            ┌─────────────────────────────────┐
            │    Le classi sono bilanciate?   │
            │         (~50/50)                │
            └─────────────────┬───────────────┘
                              │
              ┌───────────────┴───────────────┐
              │                               │
              ▼                               ▼
            [ Sì ]                          [ No ]
              │                               │
              ▼                               ▼
        ┌───────────┐              ┌──────────────────┐
        │ Accuracy  │              │ Precision/Recall │
        │   o F1    │              │  F1 o PR-AUC     │
        └───────────┘              └────────┬─────────┘
                                            │
                          ┌─────────────────┴─────────────────┐
                          │                                   │
                          ▼                                   ▼
                   FN è costoso?                        FP è costoso?
                   (es: diagnosi)                       (es: spam)
                          │                                   │
                          ▼                                   ▼
                    ┌───────────┐                      ┌────────────┐
                    │  RECALL   │                      │ PRECISION  │
                    │ priorità  │                      │ priorità   │
                    └───────────┘                      └────────────┘
```

## 📋 Tabella Riassuntiva Metriche

| Metrica | Formula | Quando Usare |
|---------|---------|--------------|
| **Accuracy** | (TP+TN)/Tot | Classi bilanciate |
| **Precision** | TP/(TP+FP) | FP è costoso |
| **Recall** | TP/(TP+FN) | FN è costoso |
| **F1-Score** | 2PR/(P+R) | Trade-off P/R |
| **ROC-AUC** | Area ROC | Confronto modelli |
| **PR-AUC** | Area PR | Classi sbilanciate |

# Section 7 — Esercizi Svolti

---

## 🎯 Esercizio 17.1 — Analisi Completa con Classification Report

**Obiettivo**: Usare `classification_report` di sklearn e interpretare i risultati.

**Consegna**:
1. Carica il dataset Breast Cancer
2. Addestra un modello (a scelta)
3. Genera il classification report completo
4. Identifica la classe più difficile da predire
5. Calcola manualmente una delle metriche per verificare

In [ ]:
# ============================================
# ESERCIZIO 17.1 — SOLUZIONE COMPLETA
# Analisi Completa con Classification Report
# ============================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (classification_report, confusion_matrix, 
                             ConfusionMatrixDisplay, precision_score, 
                             recall_score, f1_score)
import warnings
warnings.filterwarnings('ignore')

print("=" * 60)
print("ESERCIZIO 17.1 — Classification Report Completo")
print("=" * 60)

# 1. Caricamento dataset
data = load_breast_cancer()
X = data.data
y = data.target
target_names = data.target_names  # ['malignant', 'benign']

print(f"\n📊 Dataset: Breast Cancer Wisconsin")
print(f"   - Campioni: {len(X)}")
print(f"   - Features: {X.shape[1]}")
print(f"   - Classi: {target_names}")
print(f"   - Distribuzione: Maligno={sum(y==0)}, Benigno={sum(y==1)}")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 2. Training modello
rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

print("\n✅ Random Forest addestrato")

# 3. Classification Report
print("\n" + "=" * 60)
print("📊 CLASSIFICATION REPORT")
print("=" * 60)
print(classification_report(y_test, y_pred, target_names=target_names))

# 4. Analisi per classe
print("=" * 60)
print("🔍 ANALISI PER CLASSE")
print("=" * 60)

cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cm.ravel()

# Classe 0 = Malignant (è la "positiva" dal punto di vista medico!)
# Classe 1 = Benign

print("\n📌 Classe MALIGNO (0):")
prec_mal = tn / (tn + fn)  # Precision per classe 0
rec_mal = tn / (tn + fp)   # Recall per classe 0
print(f"   - I falsi negativi (FN) per maligno = {fp}")
print(f"   - Questi sono tumori maligni classificati come benigni! GRAVE!")

print("\n📌 Classe BENIGNO (1):")
prec_ben = tp / (tp + fp)
rec_ben = tp / (tp + fn)
print(f"   - I falsi positivi (FP) per benigno = {fn}")
print(f"   - Questi sono tumori benigni classificati come maligni")
print(f"   - Meno grave, ma causa ansia inutile")

# 5. Calcolo manuale per verifica
print("\n" + "=" * 60)
print("🧮 VERIFICA CALCOLO MANUALE (Classe 1 = Benigno)")
print("=" * 60)

precision_manual = tp / (tp + fp)
recall_manual = tp / (tp + fn)
f1_manual = 2 * precision_manual * recall_manual / (precision_manual + recall_manual)

precision_sklearn = precision_score(y_test, y_pred)
recall_sklearn = recall_score(y_test, y_pred)
f1_sklearn = f1_score(y_test, y_pred)

print(f"\n   {'Metrica':<15} {'Manuale':<12} {'Sklearn':<12} {'Match':<8}")
print("-" * 55)
print(f"   {'Precision':<15} {precision_manual:<12.6f} {precision_sklearn:<12.6f} {'✅' if abs(precision_manual - precision_sklearn) < 0.0001 else '❌'}")
print(f"   {'Recall':<15} {recall_manual:<12.6f} {recall_sklearn:<12.6f} {'✅' if abs(recall_manual - recall_sklearn) < 0.0001 else '❌'}")
print(f"   {'F1-Score':<15} {f1_manual:<12.6f} {f1_sklearn:<12.6f} {'✅' if abs(f1_manual - f1_sklearn) < 0.0001 else '❌'}")

# Visualizzazione
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Confusion matrix
ax1 = axes[0]
disp = ConfusionMatrixDisplay(cm, display_labels=target_names)
disp.plot(ax=ax1, cmap='Blues')
ax1.set_title('📊 Matrice di Confusione')

# Metriche per classe
ax2 = axes[1]
metriche_per_classe = pd.DataFrame({
    'Precision': [precision_score(y_test, y_pred, pos_label=0), 
                  precision_score(y_test, y_pred, pos_label=1)],
    'Recall': [recall_score(y_test, y_pred, pos_label=0), 
               recall_score(y_test, y_pred, pos_label=1)],
    'F1': [f1_score(y_test, y_pred, pos_label=0), 
           f1_score(y_test, y_pred, pos_label=1)]
}, index=target_names)

metriche_per_classe.plot(kind='bar', ax=ax2, color=['#3498db', '#2ecc71', '#9b59b6'], 
                          edgecolor='black', alpha=0.8)
ax2.set_title('📈 Metriche per Classe')
ax2.set_ylabel('Valore')
ax2.set_ylim(0, 1.1)
ax2.legend(loc='lower right')
ax2.set_xticklabels(ax2.get_xticklabels(), rotation=0)

plt.tight_layout()
plt.show()

# Conclusione
print("\n💡 Conclusione:")
print(f"   In questo contesto medico, un FN (maligno → benigno) è MOLTO grave!")
print(f"   FN attuali: {fp} casi di cancro non rilevati")
print(f"   Potremmo abbassare la soglia per aumentare il Recall sulla classe maligno.")

---

## 🎯 Esercizio 17.2 — Confronto Modelli con ROC-AUC

**Obiettivo**: Confrontare più modelli usando la curva ROC.

**Consegna**:
1. Addestra 3 modelli diversi sullo stesso dataset
2. Calcola ROC-AUC per ciascuno
3. Plotta le 3 curve ROC insieme
4. Identifica il modello migliore
5. Verifica se il modello migliore per AUC è anche il migliore per F1

In [ ]:
# ============================================
# ESERCIZIO 17.2 — SOLUZIONE COMPLETA
# Confronto Modelli con ROC-AUC
# ============================================

import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import roc_curve, roc_auc_score, f1_score
import warnings
warnings.filterwarnings('ignore')

print("=" * 60)
print("ESERCIZIO 17.2 — Confronto Modelli con ROC-AUC")
print("=" * 60)

# 1. Generazione dataset
X, y = make_classification(
    n_samples=1000,
    n_features=20,
    n_informative=10,
    n_redundant=5,
    weights=[0.7, 0.3],  # Leggermente sbilanciato
    random_state=42
)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(f"\n📊 Dataset generato:")
print(f"   - Campioni: {len(X)}")
print(f"   - Features: {X.shape[1]}")
print(f"   - Classe 0: {(y==0).sum()}, Classe 1: {(y==1).sum()}")

# 2. Definizione modelli
modelli = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
    'SVM (RBF)': SVC(kernel='rbf', probability=True, random_state=42)
}

# 3. Training e calcolo metriche
risultati = []
colors = ['#3498db', '#2ecc71', '#e74c3c', '#9b59b6']

plt.figure(figsize=(10, 7))

for (nome, modello), color in zip(modelli.items(), colors):
    # Training
    modello.fit(X_train, y_train)
    
    # Probabilità
    y_proba = modello.predict_proba(X_test)[:, 1]
    y_pred = modello.predict(X_test)
    
    # Metriche
    auc = roc_auc_score(y_test, y_proba)
    f1 = f1_score(y_test, y_pred)
    
    risultati.append({
        'Modello': nome,
        'AUC': auc,
        'F1': f1,
        'color': color
    })
    
    # ROC curve
    fpr, tpr, _ = roc_curve(y_test, y_proba)
    plt.plot(fpr, tpr, label=f'{nome} (AUC = {auc:.3f})', linewidth=2, color=color)

# Linea random
plt.plot([0, 1], [0, 1], 'k--', linewidth=1.5, label='Random (AUC = 0.5)')

plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('📈 Confronto Curve ROC - 4 Modelli', fontsize=14)
plt.legend(loc='lower right', fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# 4. Tabella risultati
print("\n📊 RISULTATI:")
print("-" * 60)
print(f"{'Modello':<25} {'AUC-ROC':>12} {'F1-Score':>12}")
print("-" * 60)

risultati_df = pd.DataFrame(risultati)
risultati_df = risultati_df.sort_values('AUC', ascending=False)

for _, row in risultati_df.iterrows():
    print(f"{row['Modello']:<25} {row['AUC']:>12.4f} {row['F1']:>12.4f}")

# 5. Identificazione vincitore
best_auc = risultati_df.iloc[0]
best_f1 = risultati_df.sort_values('F1', ascending=False).iloc[0]

print("\n" + "=" * 60)
print("🏆 VINCITORI")
print("=" * 60)
print(f"\n   Migliore per AUC-ROC: {best_auc['Modello']} ({best_auc['AUC']:.4f})")
print(f"   Migliore per F1-Score: {best_f1['Modello']} ({best_f1['F1']:.4f})")

if best_auc['Modello'] == best_f1['Modello']:
    print("\n   ✅ Lo stesso modello vince per entrambe le metriche!")
else:
    print("\n   ⚠️ Modelli diversi! Questo può succedere:")
    print("   - AUC misura la capacità di ranking (su tutte le soglie)")
    print("   - F1 misura la performance a soglia 0.5")

# Grafico confronto finale
fig, ax = plt.subplots(figsize=(10, 5))
x = np.arange(len(risultati_df))
width = 0.35

bars1 = ax.bar(x - width/2, risultati_df['AUC'], width, label='AUC-ROC', 
               color='#3498db', edgecolor='black', alpha=0.8)
bars2 = ax.bar(x + width/2, risultati_df['F1'], width, label='F1-Score', 
               color='#e74c3c', edgecolor='black', alpha=0.8)

ax.set_ylabel('Score')
ax.set_title('📊 Confronto AUC vs F1 per Modello')
ax.set_xticks(x)
ax.set_xticklabels(risultati_df['Modello'], rotation=15, ha='right')
ax.legend()
ax.set_ylim(0, 1.1)

for bar in bars1:
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
            f'{bar.get_height():.3f}', ha='center', fontsize=9)
for bar in bars2:
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
            f'{bar.get_height():.3f}', ha='center', fontsize=9)

plt.tight_layout()
plt.show()

---

## 🎯 Esercizio 17.3 — Ottimizzazione Soglia per Business Case

**Obiettivo**: Trovare la soglia che minimizza il costo totale in uno scenario reale.

**Consegna**:
1. Simula uno scenario di rilevamento frodi:
   - Costo FP (blocco transazione legittima): $20
   - Costo FN (frode non rilevata): $500
2. Calcola il costo totale per diverse soglie
3. Trova la soglia che minimizza il costo
4. Confronta con la soglia di default (0.5)
5. Visualizza il risparmio

In [ ]:
# ============================================
# ESERCIZIO 17.3 — SOLUZIONE COMPLETA
# Ottimizzazione Soglia per Business Case
# ============================================

import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')

print("=" * 60)
print("ESERCIZIO 17.3 — Ottimizzazione Soglia per Business Case")
print("=" * 60)

# Scenario: Rilevamento Frodi
print("""
📌 SCENARIO: Sistema di Rilevamento Frodi
─────────────────────────────────────────
• Classe 0: Transazione legittima
• Classe 1: Frode (positivo)

COSTI:
• FP (blocco transazione legittima): $20 (fastidio cliente)
• FN (frode non rilevata): $500 (perdita effettiva)
""")

COSTO_FP = 20   # Blocco transazione legittima
COSTO_FN = 500  # Frode non rilevata

# 1. Generazione dataset (frodi = 5%)
X, y = make_classification(
    n_samples=5000,
    n_features=15,
    n_informative=10,
    weights=[0.95, 0.05],  # 5% frodi
    random_state=42
)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(f"📊 Dataset:")
print(f"   - Transazioni totali: {len(X)}")
print(f"   - Frodi: {(y==1).sum()} ({(y==1).mean():.1%})")
print(f"   - Test set: {len(y_test)} transazioni")

# 2. Training modello
rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)
y_proba = rf.predict_proba(X_test)[:, 1]

print("\n✅ Modello addestrato")

# 3. Calcolo costo per diverse soglie
soglie = np.arange(0.01, 0.99, 0.01)
costi = []
dettagli = []

for soglia in soglie:
    y_pred = (y_proba >= soglia).astype(int)
    
    fp = ((y_pred == 1) & (y_test == 0)).sum()  # Legittime bloccate
    fn = ((y_pred == 0) & (y_test == 1)).sum()  # Frodi non rilevate
    
    costo_totale = fp * COSTO_FP + fn * COSTO_FN
    costi.append(costo_totale)
    dettagli.append({'soglia': soglia, 'fp': fp, 'fn': fn, 'costo': costo_totale})

# 4. Trova soglia ottimale
dettagli_df = pd.DataFrame(dettagli)
idx_ottimo = np.argmin(costi)
soglia_ottima = soglie[idx_ottimo]
costo_minimo = costi[idx_ottimo]

# Costo a soglia 0.5 (default)
idx_default = np.where(soglie >= 0.5)[0][0]
costo_default = costi[idx_default]

print("\n" + "=" * 60)
print("📊 RISULTATI OTTIMIZZAZIONE")
print("=" * 60)

print(f"\n🎯 Soglia ottimale: {soglia_ottima:.2f}")
print(f"   Costo totale: ${costo_minimo:,.0f}")
print(f"   FP (legit bloccate): {dettagli_df.iloc[idx_ottimo]['fp']:.0f}")
print(f"   FN (frodi perse): {dettagli_df.iloc[idx_ottimo]['fn']:.0f}")

print(f"\n📌 Soglia default (0.5):")
print(f"   Costo totale: ${costo_default:,.0f}")
print(f"   FP: {dettagli_df.iloc[idx_default]['fp']:.0f}")
print(f"   FN: {dettagli_df.iloc[idx_default]['fn']:.0f}")

risparmio = costo_default - costo_minimo
risparmio_perc = (risparmio / costo_default) * 100

print(f"\n💰 RISPARMIO con soglia ottimizzata:")
print(f"   ${risparmio:,.0f} ({risparmio_perc:.1f}%)")

# 5. Visualizzazione
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Costo vs Soglia
ax1 = axes[0]
ax1.plot(soglie, costi, linewidth=2, color='#e74c3c')
ax1.axvline(x=soglia_ottima, color='green', linestyle='--', linewidth=2, 
            label=f'Ottima: {soglia_ottima:.2f}')
ax1.axvline(x=0.5, color='gray', linestyle=':', linewidth=2, 
            label='Default: 0.50')
ax1.scatter([soglia_ottima], [costo_minimo], s=150, color='green', zorder=5)
ax1.scatter([0.5], [costo_default], s=150, color='gray', zorder=5)

ax1.set_xlabel('Soglia di Classificazione', fontsize=12)
ax1.set_ylabel('Costo Totale ($)', fontsize=12)
ax1.set_title('💰 Costo Totale vs Soglia', fontsize=14)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Breakdown FP vs FN
ax2 = axes[1]
ax2.plot(soglie, dettagli_df['fp'] * COSTO_FP, linewidth=2, 
         label=f'Costo FP (${COSTO_FP}/blocco)', color='#3498db')
ax2.plot(soglie, dettagli_df['fn'] * COSTO_FN, linewidth=2, 
         label=f'Costo FN (${COSTO_FN}/frode)', color='#e74c3c')
ax2.axvline(x=soglia_ottima, color='green', linestyle='--', linewidth=2, 
            label=f'Soglia ottima: {soglia_ottima:.2f}')

ax2.set_xlabel('Soglia di Classificazione', fontsize=12)
ax2.set_ylabel('Costo ($)', fontsize=12)
ax2.set_title('📊 Breakdown: Costo FP vs FN', fontsize=14)
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Tabella confronto
print("\n" + "=" * 60)
print("📋 TABELLA CONFRONTO")
print("=" * 60)
print(f"""
┌──────────────┬──────────────┬──────────────┬──────────────┐
│    Soglia    │   Costo FP   │   Costo FN   │ Costo Totale │
├──────────────┼──────────────┼──────────────┼──────────────┤
│ 0.50 (def)   │ ${dettagli_df.iloc[idx_default]['fp'] * COSTO_FP:>10,.0f} │ ${dettagli_df.iloc[idx_default]['fn'] * COSTO_FN:>10,.0f} │ ${costo_default:>10,.0f} │
│ {soglia_ottima:.2f} (opt)   │ ${dettagli_df.iloc[idx_ottimo]['fp'] * COSTO_FP:>10,.0f} │ ${dettagli_df.iloc[idx_ottimo]['fn'] * COSTO_FN:>10,.0f} │ ${costo_minimo:>10,.0f} │
├──────────────┼──────────────┼──────────────┼──────────────┤
│ RISPARMIO    │              │              │ ${risparmio:>10,.0f} │
└──────────────┴──────────────┴──────────────┴──────────────┘
""")

print("💡 Conclusione:")
print(f"   Abbassando la soglia da 0.50 a {soglia_ottima:.2f}, accettiamo")
print(f"   più falsi positivi (blocchi errati) ma riduciamo le frodi perse,")
print(f"   risparmiando ${risparmio:,.0f} complessivamente!")

---

## 📌 Conclusione e Bignami

### ✅ Cosa Abbiamo Imparato

1. **L'accuracy non basta** — su classi sbilanciate è fuorviante
2. **Precision vs Recall** — trade-off inevitabile, scegli in base al contesto
3. **La soglia conta** — non usare sempre 0.5, ottimizzala per il business
4. **ROC-AUC vs PR-AUC** — PR-AUC più informativa su dataset sbilanciati
5. **Costi reali** — quando FP e FN hanno costi diversi, ottimizza per il costo totale

---

## 📋 BIGNAMI — Metriche di Classificazione

| Metrica | Formula | Quando Usarla | Esempio |
|---------|---------|---------------|---------|
| **Accuracy** | (TP+TN)/(Totale) | Classi bilanciate | Classificazione generi |
| **Precision** | TP/(TP+FP) | FP costosi | Spam, raccomandazioni |
| **Recall** | TP/(TP+FN) | FN costosi | Malattie, frodi |
| **F1-Score** | 2·(P·R)/(P+R) | Bilanciare P e R | Generale |
| **Specificity** | TN/(TN+FP) | Veri negativi importanti | Test medici |
| **ROC-AUC** | Area sotto ROC | Confronto modelli | Benchmark |
| **PR-AUC** | Area sotto PR | Classi sbilanciate | Frodi, malattie rare |

---

### 🎯 Flowchart Decisionale Rapido

```
Dataset bilanciato? → Accuracy + ROC-AUC
          ↓ NO
Classi sbilanciate? → PR-AUC + F1 della classe rara
          ↓
FN più costosi di FP? → Massimizza Recall (abbassa soglia)
FP più costosi di FN? → Massimizza Precision (alza soglia)
Costi noti? → Minimizza Costo = FP×C_FP + FN×C_FN
```

---

### 🔧 Codice Rapido di Riferimento

```python
from sklearn.metrics import (
    confusion_matrix, classification_report,
    precision_score, recall_score, f1_score,
    roc_curve, roc_auc_score,
    precision_recall_curve, average_precision_score
)

# Report completo
print(classification_report(y_true, y_pred))

# Curva ROC
fpr, tpr, thresholds = roc_curve(y_true, y_proba)
auc = roc_auc_score(y_true, y_proba)

# Soglia ottimale (Youden's J)
j_scores = tpr - fpr
soglia_ottima = thresholds[np.argmax(j_scores)]
```

---

### ⚠️ Errori Comuni da Evitare

| ❌ Errore | ✅ Soluzione |
|-----------|-------------|
| Usare accuracy su dati sbilanciati | Usa F1, PR-AUC |
| Ignorare la soglia | Ottimizzala per il business |
| Confrontare ROC su dati sbilanciati | Usa PR curve |
| Non calcolare entrambe P e R | Guarda sempre il trade-off |
| Non considerare i costi | Quantifica FP e FN in € |

---

### 🚀 Prossimi Passi
Nella **Lezione 18** vedremo il **Clustering** — algoritmi non supervisionati per scoprire gruppi naturali nei dati!